In [37]:
import re
import itertools
import pandas as pd
from tqdm import tqdm
import pickle
import json
import jellyfish
from cdifflib import CSequenceMatcher
import textdistance
import editdistance
import fuzzy
from collections import Counter
from cdifflib import CSequenceMatcher
from fuzzywuzzy import fuzz

In [38]:
ment=pd.read_csv("index_files4/final_mod_ment_w_baseline_gen4_1.csv",sep=",")

In [39]:
ment.tail()

,advisorId,researcherId,advisor_name,researcher_name,dc.date.submitted[],dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,...,dc.subject.ddc,dc.source.uri,dc.description.abstract,dc.title.alternative,advisor_inst_dept,stud_inst_dept,adv_names_rid,res_names_rid,advId,resId
205230,283495,280288,"Vivekanandan, P.","K, Parkavi",2017-01-01,eng,faculty of information and communication engin...,D334,Anna University,I250,...,"Computer science, information & general works,...",http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Vivekanandan, P.@I250@D334","K, Parkavi@I250@D334","Vivekanandan, P.@283495","K, Parkavi@280288",283495,280288
205231,283827,170514,"Hire, Pramodkumar S.","Patil, Archana Dilip.",2017-08-31,eng,department of geography,D2266,Tilak Maharashtra Vidyapeeth,I13,...,History & geography,http://shodhganga.inflibnet.ac.in/,NaN,ts forms and processes,"Hire, Pramodkumar S.@I13@D2266","Patil, Archana Dilip.@I13@D2266","Hire, Pramodkumar S.@283827","Patil, Archana Dilip.@170514",283827,170514
205232,283901,164072,"Kankure, Kishan Babarao","Kore, Nirmala Sopan",2002-01-01,eng,department of geography,D2266,Swami Ramanand Teerth Marathwada University,I295,...,History & geography,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Kankure, Kishan Babarao@I295@D2266","Kore, Nirmala Sopan@I295@D2266","Kankure, Kishan Babarao@283901","Kore, Nirmala Sopan@164072",283901,164072
205233,283901,164083,"Kankure, Kishan Babarao","Ugade, Vasant Ramrao",2005-01-01,eng,department of geography,D2266,Swami Ramanand Teerth Marathwada University,I295,...,History & geography,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Kankure, Kishan Babarao@I295@D2266","Ugade, Vasant Ramrao@I295@D2266","Kankure, Kishan Babarao@283901","Ugade, Vasant Ramrao@164083",283901,164083
205234,283928,96030,"Disouza, John Intru","Jamkhandi, Chemansaheb Meerasaheb",2013-10-09,eng,faculty of pharmacy,D1373,Shri Jagdishprasad Jhabarmal Tibarewala Univer...,I17,...,"Technology,Medicine & health,Pharmacology and ...",http://shodhganga.inflibnet.ac.in/,The objective of the present study was to eval...,NaN,"Disouza, John Intru@I17@D1373","Jamkhandi, Chemansaheb Meerasaheb@I17@D1373","Disouza, John Intru@283928","Jamkhandi, Chemansaheb Meerasaheb@96030",283928,96030


In [40]:
chars=[",",";","\.","(",")"]

In [41]:
def rm_splChar(name):
    name = str(name)
    name1 = re.sub(" +","",name)
    regex = "|".join(chars)
    name1 = re.sub(regex,"", name1)
    name2 = re.sub(regex,"", name)
    #val = re.sub('[^A-Za-z]+', '', val)
    return name1, name2

In [42]:
def char_dist(name1, name2):
    name1=rm_splChar(name1)
    name2=rm_splChar(name2)
    return int(Counter(name1)==Counter(name2))

In [43]:
def diff_lib(name1, name2):
    name1=name1.lower()
    name2=name2.lower()
    ratio=CSequenceMatcher(lambda x: x == ' ', name1, name2).ratio()
    return ratio

In [44]:
def jaro_winkler_score(name1, name2):
    jw_score=textdistance.jaro_winkler.normalized_similarity(name1,name2)
    return jw_score

In [45]:
def levenshtein_score(name1, name2):
    leven_score = textdistance.levenshtein.normalized_similarity(name1,name2)
    return leven_score

In [46]:
def fuzzy_nysiis(name1, name2):
    ny1=fuzzy.nysiis(name1)
    ny2=fuzzy.nysiis(name2)
    if (ny1 or ny2):
        nysiis_score = editdistance.eval(ny1, ny2)/max(len(ny1),len(ny2))
    else:
        nysiis_score = 0
    return nysiis_score

In [47]:
def fuzzy_DMetaphone(name1, name2):
    d1=jellyfish.metaphone(name1)
    d2=jellyfish.metaphone(name2)
    if (d1 or d2):
        meta_score = editdistance.eval(d1, d2)/max(len(d1),len(d2))
    else:
        meta_score = 0
    return meta_score

In [48]:
#Soundex is a phonetic algorithm 
def jellyfish_soundex(name1, name2):
    s1=jellyfish.soundex(name1)
    s2=jellyfish.soundex(name2)
    sound_score = editdistance.eval(s1,s2)/max(len(s1),len(s2))
    return sound_score

In [49]:
def fuzzy_wuzzy(name1, name2):
    fuzz_score=fuzz.token_set_ratio(name1, name2)/100
    return fuzz_score

In [50]:
def hamming_similarity(name1, name2):
    h_score=textdistance.hamming.normalized_similarity(name1,name2)
    return h_score

In [51]:
def jaccard_similarity(name1, name2):
    j_score=textdistance.jaccard.normalized_similarity(name1,name2)
    return j_score

In [52]:
def cosine_similarity(name1, name2):
    c_score=textdistance.jaccard.normalized_similarity(name1,name2)
    return c_score

In [53]:
def damerau_levenshtein_similarity(name1, name2):
    dl_score=textdistance.damerau_levenshtein.normalized_similarity(name1,name2)
    return dl_score

In [54]:
def sorensen_dice_similarity(name1, name2):
    sd_score=textdistance.sorensen_dice.normalized_similarity(name1,name2)
    return sd_score

In [55]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [56]:
def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [57]:
def calculate_feats(name1, name2):
    sim_score=[]
    sim_score.append(diff_lib(name1,name2))
    sim_score.append(jaro_winkler_score(name1, name2))
    sim_score.append(levenshtein_score(name1, name2))
    sim_score.append(1-fuzzy_nysiis(name1, name2))#distance
    sim_score.append(1-fuzzy_DMetaphone(name1, name2))#distance
    sim_score.append(1-jellyfish_soundex(name1,name2))#distance
    sim_score.append(fuzzy_wuzzy(name1, name2))
    sim_score.append(hamming_similarity(name1, name2))
    sim_score.append(jaccard_similarity(name1, name2))
    sim_score.append(cosine_similarity(name1, name2))
    sim_score.append(damerau_levenshtein_similarity(name1, name2))
    sim_score.append(sorensen_dice_similarity(name1, name2))
    return sim_score

In [58]:
def lcs(name1, name2):
    match = CSequenceMatcher(None, name1, name2).find_longest_match(0, len(name1), 0, len(name2))
    common_subs=name1[match.a: match.a + match.size]
    name1=re.sub(re.escape(common_subs),"",name1)
    name2=re.sub(re.escape(common_subs),"",name2)
    return name1,name2

In [59]:
#n101, n201 = lcs("Johri, Alok","Alok, Johri.")

In [60]:
#sum(calculate_feats(n101.lower(),n201.lower()))

In [61]:
def find_similar_names(org_df, names_df):
    similar_names = {}
    succ=0
    keep_cnt = 0
    count = 0
    try:
        for name1, name2 in tqdm(itertools.combinations(names_df, 2), total=(names_df.shape[0]*(names_df.shape[0]-1))/2):
            keep_cnt += 1
            n1, n10 = rm_splChar(name1)
            n2, n20 = rm_splChar(name2)
            check=len(set(name1.split()).intersection(name2.split()))/max(len(name1),len(name2))
            if check > 0.02:
                n11, n21 = lcs(n1, n2)
                n101, n201 = lcs(n10,n20)
                if (n11 or n21): 
                    vec2 = calculate_feats(n11.lower(), n21.lower())
                else:
                    vec2 = calculate_feats(n1.lower(), n2.lower())
                if (n101 or n201): 
                    #vec3=calculate_feats(n101, n201)
                    vec4=calculate_feats(n101.lower(), n201.lower())
                else:
                    #vec3=calculate_feats(n10, n20)
                    vec4=calculate_feats(n10.lower(), n20.lower())
                if (sum(vec2) > 9) or (sum(vec4) > 9) :
                    inst1 = org_df[(org_df['publisher_dept'] == name1)]["instituteId"]
                    inst2 = org_df[(org_df['publisher_dept'] == name2)]["instituteId"]
                    common_inst=set(inst1).intersection(inst2)
                    if common_inst :
                        dept1_id=pd.unique(org_df[(org_df['publisher_dept'] == name1)]["DepartmentId"])
                        dept2_id=pd.unique(org_df[(org_df['publisher_dept'] == name2)]["DepartmentId"])
                        similar_names[(tuple(dept1_id),tuple(dept2_id))]=(name1,name2,sum(vec2), sum(vec4))
                        succ+=1
                    count += 1
    except KeyboardInterrupt:
        print('keyboard error')
    except ZeroDivisionError:
        print('divided by zero error')
    except Exception as e:
        print(e)
    finally:
        save_obj(similar_names, "disambigauted_dept_wid_"+str(keep_cnt))
        print("Pairs having similarity value greater than 10: "+str(count))
        print("Successful Pairs: "+str(succ))
        return similar_names

In [62]:
ment['publisher_dept']=ment['publisher_dept'].fillna(" ")

In [ ]:
calculate_feats(n11.lower(), n21.lower())

In [161]:
#ment=ment[ment['advisor_name'].str.contains('Ajith')]

In [63]:
unique_dept = pd.unique(ment['publisher_dept'])

In [64]:
sim_dept=find_similar_names(ment,unique_dept) # Read file alreday generated with name ,"disambigauted_dept_3991725.pkl"

100%|██████████| 3991725/3991725.0 [2:25:36<00:00, 456.90it/s]  


Pairs having similarity value greater than 10: 643
Successful Pairs: 211


In [27]:
sim_dept=load_obj('disambigauted_dept_3991725.pkl')

In [103]:
sim_dept

{3: ('department of sacred heart college(thevara). department of botany',
  'department of sacred heart college(thevara).department of botany',
  12.0,
  9.994870305845915),
 5652: ('department of sacred heart college(thevara). department of economics and research centre',
  'department of sacred heart college(thevara).departments of economics and research centre',
  10.317351294621497,
  10.629303428692964),
 6992: ('department of sacred heart college(thevara). department of economics and research centre',
  'department of sree sankara college(kalady). department of commerce and research centre',
  9.051233131307365,
  9.093886475296827),
 15456: ('department of sacred heart college(thevara).departments of economics and research centre',
  'department of sree sankara college(kalady).department of economics and research centre',
  8.934832770509185,
  9.13600216357437),
 39607: ('department of plant biotechnology',
  'department of biotechnology',
  9.372427315753889,
  7.4323422965063

In [71]:
#f_sim_names1=[sim_dept[a] for a in sim_dept if (sim_dept[a][2] >= 12 or sim_dept[a][3] >= 12) or (sim_dept[a][2] >= 10 and sim_dept[a][3] >= 10)] 

In [ ]:
#f_sim_dept = 

In [28]:
remove = [('department of mechanical engineering','department of chemical engineering'),('department of organic chemistry','department of inorganic chemistry'),('department of chemical engineering','department of mechanicalengineering'),('department of chemical oceanography','department of physical oceanography')]

In [29]:
f_sim_names=[(sim_dept[a][0],sim_dept[a][1]) for a in sim_dept if (sim_dept[a][2] >= 10 or sim_dept[a][3] >= 10)]

In [30]:
f_sim_names=[a for a in f_sim_names if a not in remove]

In [31]:
len(f_sim_names)

79

In [32]:
ment['N_DepartmentId']=ment['DepartmentId']
tmp_dict={}
for a in f_sim_names:
    dept1=ment[ment['publisher_dept']==tmp_dict[a[0]]]['N_DepartmentId'].values[0]
    dept2=ment[ment['publisher_dept']==a[1]]['N_DepartmentId'].values[0]
    if dept1 in tmp_dict:
        ment.loc[(ment['publisher_dept'].isin([a[0],a[1]]),'N_DepartmentId']= tmp_dict[dept1]
    elif dept2 in tmp_dict:
        ment.loc[(ment['publisher_dept'].isin([a[0],a[1]]),'N_DepartmentId']= dept1
    else:
        tmp_dict[dept2]=dept1

In [33]:
ment.to_csv("index_files4/final_mod_ment_w_baseline_gen4_2.csv",sep=",",index=False)

In [34]:
ment[ment['publisher_dept']=='department of sacred heart college(thevara). department of botany']

,advisorId,researcherId,advisor_name,researcher_name,dc.date.submitted[],dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,...,dc.source.uri,dc.description.abstract,dc.title.alternative,advisor_inst_dept,stud_inst_dept,adv_names_rid,res_names_rid,advId,resId,N_DepartmentId
201300,132735,95370,"Jose, Joseph","Thoppil, John E.",1998-01-01,eng,department of sacred heart college(thevara). d...,D1321,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Jose, Joseph@I251@D1321","Thoppil, John E.@I251@D1321","Jose, Joseph@132735","Thoppil, John E.@95370",132735,95370,D1321


In [35]:
ment[ment['publisher_dept']=='department of sacred heart college(thevara).department of botany']

,advisorId,researcherId,advisor_name,researcher_name,dc.date.submitted[],dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,...,dc.source.uri,dc.description.abstract,dc.title.alternative,advisor_inst_dept,stud_inst_dept,adv_names_rid,res_names_rid,advId,resId,N_DepartmentId
48,26,95416,"Joy, C. M.","Jose, K P.",2002-01-07,eng,department of sacred heart college(thevara).de...,D129,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Joy, C. M.@I251@D129","Jose, K P.@I251@D129","Joy, C. M.@26","Jose, K P.@95416",26,95416,D1321
50,28,95418,"Joseph, Joy P.","Aruna, A.",2004-01-04,eng,department of sacred heart college(thevara).de...,D129,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Joseph, Joy P.@I251@D129","Aruna, A.@I251@D129","Joseph, Joy P.@28","Aruna, A.@95418",28,95418,D1321
201301,132735,95425,"Jose, Joseph","Joseph, Roy",1998-01-01,eng,department of sacred heart college(thevara).de...,D129,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Jose, Joseph@I251@D129","Joseph, Roy@I251@D129","Jose, Joseph@132735","Joseph, Roy@95425",132735,95425,D1321


In [60]:
ment[ment['advisor_name'].str.contains('Ajith')]#['publisher_dept'].values

,advisorId,researcherId,advisor_name,researcher_name,dc.date.submitted[],dc.language.iso[],publisher_dept,DepartmentId,publisher_institution,instituteId,...,dc.source.uri,dc.description.abstract,dc.title.alternative,advisor_inst_dept,stud_inst_dept,adv_names_rid,res_names_rid,advId,resId,N_DepartmentId
19,9,95381,"Kumar, N. Ajith","Siji, S R.",2013-03-13,eng,department of sacred heart college,D2490,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,The present study is an attempt to examine the...,NaN,"Kumar, N. Ajith@I251@D2490","Siji, S R.@I251@D2490","Kumar, N. Ajith@9","Siji, S R.@95381",17,95381,D2490
20,9,95388,"Kumar, N. Ajith","Liji, K T.",2013-04-01,eng,department of sacred heart college,D2490,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Kumar, N. Ajith@I251@D2490","Liji, K T.@I251@D2490","Kumar, N. Ajith@9","Liji, K T.@95388",17,95388,D2490
21,9,95394,"Kumar, N. Ajith","Thachil, Preemy P.",2008-12-12,eng,department of sacred heart college,D2490,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Kumar, N. Ajith@I251@D2490","Thachil, Preemy P.@I251@D2490","Kumar, N. Ajith@9","Thachil, Preemy P.@95394",17,95394,D2490
22,9,95406,"Kumar, N. Ajith","Kumar, C Ajaya",2011-11-22,eng,department of sacred heart college,D2490,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Kumar, N. Ajith@I251@D2490","Kumar, C Ajaya@I251@D2490","Kumar, N. Ajith@9","Kumar, C Ajaya@95406",17,95406,D2490
23,9,95410,"Kumar, N. Ajith","Varma, P R Swathy",2012-01-27,eng,department of sacred heart college,D2490,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Kumar, N. Ajith@I251@D2490","Varma, P R Swathy@I251@D2490","Kumar, N. Ajith@9","Varma, P R Swathy@95410",17,95410,D2490
25,12,95387,"Ajith Kumar, N.","Mathew, K J.",2012-03-29,eng,department of sacred heart college,D2490,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Ajith Kumar, N.@I251@D2490","Mathew, K J.@I251@D2490","Ajith Kumar, N.@12","Mathew, K J.@95387",17,95387,D2490
26,12,95419,"Ajith Kumar, N.","Liya, C A Mary",2014-06-17,eng,department of sacred heart college,D2490,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Ajith Kumar, N.@I251@D2490","Liya, C A Mary@I251@D2490","Ajith Kumar, N.@12","Liya, C A Mary@95419",17,95419,D2490
32,17,95393,"Ajithkumar, N.","Padmanabhan, Nirmala",2009-02-06,eng,department of sacred heart college,D2490,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Ajithkumar, N.@I251@D2490","Padmanabhan, Nirmala@I251@D2490","Ajithkumar, N.@17","Padmanabhan, Nirmala@95393",17,95393,D2490
45,24,95414,"Kumar, N. Ajith","Correya, Titus",2000-03-30,eng,department of sacred heart college(thevara). d...,D408,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Kumar, N. Ajith@I251@D408","Correya, Titus@I251@D408","Kumar, N. Ajith@24","Correya, Titus@95414",24,95414,D408
51,29,95420,"Kumar, N. Ajith","Thomas, Laisa",2005-01-08,eng,department of sacred heart college(thevara).de...,D2238,Mahatma Gandhi University,I251,...,http://shodhganga.inflibnet.ac.in/,NaN,NaN,"Kumar, N. Ajith@I251@D2238","Thomas, Laisa@I251@D2238","Kumar, N. Ajith@29","Thomas, Laisa@95420",29,95420,D408


In [59]:
pd.unique(ment['DepartmentId']).shape[0]-pd.unique(ment['N_DepartmentId']).shape[0]

62